# Fuel Prices

## Impact of the 2023 Elections in Poland

SOURCE DATA:

https://ec.europa.eu/energy/observatory/reports/

European Commission website, energy section.
We are using the oil bulletin prices spreadsheet (CTR sheet).

# Control panel:

In [1]:
# Date from which we start downloading data:
Date_start= '2018-01-01'

# Date until which we download data:
Date_end = '2023-12-31'

## Libraries

In [2]:
import pandas as pd
import numpy as np
import requests
import warnings
import os

# Data download

In [3]:
%%time
# Suppressing the warning
warnings.simplefilter("ignore")
# Disable warnings from urllib3
requests.packages.urllib3.disable_warnings()
url = "https://ec.europa.eu/energy/observatory/reports/Oil_Bulletin_Prices_History.xlsx"

# GET request to the URL and storing the response content
response = requests.get(url, verify = False)
data = response.content
# Read content into a pandas dataframe
oil_bulletin = pd.read_excel(data, sheet_name="Prices with taxes, per CTR")


CPU times: total: 3.36 s
Wall time: 6.11 s


## Data cleaning

In [4]:
#Remove the first 3 rows
oil_bulletin = oil_bulletin.iloc[3:]
# Remove "unnamed: 0" column
oil_bulletin = oil_bulletin.drop(columns = ['Unnamed: 0'])
# Remove all rows with NaN values
oil_bulletin = oil_bulletin.dropna(how='all')
oil_bulletin

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
3,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Date,Exchange\rRate\rTo €,Euro-super 95 (I),Gas oil automobile Automotive gas oil Dieselk...,Gas oil de chauffage Heating gas oil Heizöl (II),Fuel oil - Schweres Heizöl (III) Soufre,NaN,NaN
6,NaN,NaN,1000L,1000L,1000L,t,NaN,NaN
7,2024-03-18 00:00:00,1,"1,596","1,672","1,211.94",825.5,NaN,NaN
8,2024-03-11 00:00:00,1,"1,596","1,679","1,218.74",827.5,NaN,NaN
...,...,...,...,...,...,...,...,...
25162,2005-01-31 00:00:00,0.026199,888.39,881.84,432.64,196.49,191.25,518.73
25163,2005-01-24 00:00:00,0.025893,875.97,872.6,427.6,194.2,189.02,513.21
25164,2005-01-17 00:00:00,0.025901,856.56,873.65,427.74,194.26,189.08,514.92
25165,2005-01-10 00:00:00,0.025955,845.1,881.18,428.62,194.66,189.47,518.58


## Column_names mapping

In [5]:
# Mapping new column names
new_column_names = {
    'Unnamed: 1': 'Date',
    'Unnamed: 2': 'Rate to Euro',
    'Unnamed: 3': 'Euro-super 95',
    'Unnamed: 4': "Gas oil automobile",
    'Unnamed: 5': 'Automotive gas oil',
    'Unnamed: 6': 'Automotive gas oil',
    'Unnamed: 7': 'Automotive gas oil',
    'Unnamed: 8': 'LPG'
}

# Rename columns
oil_bulletin.rename(columns=new_column_names, inplace=True)

In [6]:
oil_bulletin.head()

,Date,Rate to Euro,Euro-super 95,Gas oil automobile,Automotive gas oil,Automotive gas oil,Automotive gas oil,LPG
3,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Date,Exchange\rRate\rTo €,Euro-super 95 (I),Gas oil automobile Automotive gas oil Dieselk...,Gas oil de chauffage Heating gas oil Heizöl (II),Fuel oil - Schweres Heizöl (III) Soufre,NaN,NaN
6,NaN,NaN,1000L,1000L,1000L,t,NaN,NaN
7,2024-03-18 00:00:00,1,"1,596","1,672","1,211.94",825.5,NaN,NaN
8,2024-03-11 00:00:00,1,"1,596","1,679","1,218.74",827.5,NaN,NaN


## Country mapping

> Country mapping - full country names

In [7]:
## Country mapping:
country_mapping = {
    'AT': 'Austria',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CY': 'Cyprus',
    'CZ': 'Czechia',
    'DE': 'Germany',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'GR': 'Greece',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'IE': 'Ireland',
    'IT': 'Italy',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'MT': 'Malta',
    'NL': 'Netherlands',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'SE': 'Sweden',
    'SI': 'Slovenia',
    'SK': 'Slovakia'
}

In [8]:
# Creating a mask to filter rows where 'Date' column contains 2-letter country names
mask = oil_bulletin['Date'].str.match(r'^[A-Z]{2}$').fillna(False)
# Extracting country names and converting them to a list
list_country = oil_bulletin['Date'][mask].tolist()


> Split main datframe into separate DataFrames based on country codes

In [9]:
# List where the mask is True
indices = oil_bulletin[mask].index.tolist()

# Dictionary to store new DataFrames
dfs = {} 

# Iterate over the list of indices
for i, idx in enumerate(indices):
    if i < len(indices) - 1:
        next_idx = indices[i+1]
        df_country = oil_bulletin.loc[idx:next_idx].iloc[:-1]
    else:
        df_country = oil_bulletin.loc[idx:]

    # Extract the country code from the 'Date' column 
    country_code = oil_bulletin.at[idx, 'Date']
    
    # Save to dictionary
    dfs["df_" + country_code] = df_country


In [10]:
# All df country names:
names_list = list(dfs.keys())
names_list

['df_AT',
 'df_BE',
 'df_BG',
 'df_CY',
 'df_CZ',
 'df_DE',
 'df_DK',
 'df_EE',
 'df_ES',
 'df_FI',
 'df_FR',
 'df_GR',
 'df_HR',
 'df_HU',
 'df_IE',
 'df_IT',
 'df_LT',
 'df_LU',
 'df_LV',
 'df_MT',
 'df_NL',
 'df_PL',
 'df_PT',
 'df_RO',
 'df_SE',
 'df_SI',
 'df_SK']

## DataFrame for each country

In [11]:
# Iterate over the items (key-value pairs) in the dictionary
for key, value in dfs.items():
    # Reset the index of the DataFrame to start from 0
    value = value.reset_index(drop=True)
    
    # Assign the DataFrame to a new variable
    # The variable name is derived from the key in the dictionary
    # The `globals()` function is used to assign the variable in the global scope
    globals()[key] = value

In [12]:
df_PL

,Date,Rate to Euro,Euro-super 95,Gas oil automobile,Automotive gas oil,Automotive gas oil,Automotive gas oil,LPG
0,PL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,Exchange\rRate\rTo €,Euro-super 95 (I),Gas oil automobile Automotive gas oil Dieselk...,Gas oil de chauffage Heating gas oil Heizöl (II),Fuel oil - Schweres Heizöl (III) Soufre,Fuel oil -Schweres Heizöl (III) Soufre > 1% S...,GPL pour moteur LPG motor fuel
2,NaN,NaN,1000L,1000L,1000L,t,t,1000L
3,2024-03-18 00:00:00,0.231949,"1,499.71","1,550.09","1,230.93",626.54,599.11,664.48
4,2024-03-11 00:00:00,0.233618,"1,509.99","1,564.02","1,240.57",635.72,598.67,671.62
...,...,...,...,...,...,...,...,...
952,2005-01-31 00:00:00,0.245062,893.25,827.08,436.7,177.42,143.61,456.06
953,2005-01-24 00:00:00,0.245399,883.19,827.48,444.17,163.68,144.29,455.95
954,2005-01-17 00:00:00,0.245851,877.69,832.7,432.21,169.39,133.99,466.13
955,2005-01-10 00:00:00,0.245321,875.79,838.01,420.23,168.78,140.57,469.05


# FUELS

# df_PB95

In [15]:
# Extracting Date and reset index
df_PB95 = df_PL[['Date']].copy()
df_PB95 = df_PB95.reset_index(drop=True) 

# Euro-super 95
for key, dataframe in dfs.items():
    country_code = key.split("_")[1]  
    df_PB95[country_code] = dataframe['Euro-super 95'].reset_index(drop=True) 


## Euro-super 95

In [16]:
# Keeping only datetime rows in the 'Date' column. Dropping the first 3 rows.
df_PB95['Date'] = pd.to_datetime(df_PB95['Date'], errors='coerce') 
df_PB95 = df_PB95.dropna(subset=['Date'])  



In [17]:
# Removing commas from each value:
for col in df_PB95.columns[1:]:
    df_PB95[col] = df_PB95[col].str.replace(',', '')

# Converting each column to float type:
for col in df_PB95.columns[1:]:
    df_PB95[col] = df_PB95[col].astype(float, errors='ignore')

# Dividing everything by 1000 - to get the price per liter:
df_PB95.iloc[:, 1:] = df_PB95.iloc[:, 1:] / 1000


In [18]:
# Date range
df_PB95 = df_PB95[(df_PB95['Date'] >= Date_start) & (df_PB95['Date'] <= Date_end)]

# Modifications
df_PB95 = df_PB95.reset_index(drop=True)  # Resetting index
df_PB95['Date'] = df_PB95['Date'].dt.date  # Converting the 'Date' column to date format

# Reversing the order of rows
df_PB95 = df_PB95.iloc[::-1].reset_index(drop=True)


In [19]:
# Renaming columns
df_PB95 = df_PB95.rename(columns=country_mapping)
df_PB95.head()

,Date,Austria,Belgium,Bulgaria,Cyprus,Czechia,Germany,Denmark,Estonia,Spain,...,Luxembourg,Latvia,Malta,Netherlands,Poland,Portugal,Romania,Sweden,Slovenia,Slovakia
0,2018-01-01,1.194,1.3494,1.03779,1.22350,1.19268,1.378,1.54737,1.237,1.23548,...,1.180,1.18671,1.31,1.576,1.12693,1.510,1.11676,1.49475,1.28889,1.303
1,2018-01-08,1.205,1.3617,1.04336,1.23186,1.19104,1.376,1.53357,1.245,1.24252,...,1.180,1.18671,1.31,1.581,1.13047,1.523,1.13860,1.50053,1.28889,1.305
2,2018-01-15,1.205,1.3664,1.04295,1.23281,1.19702,1.352,1.53297,1.313,1.24899,...,1.197,1.21938,1.31,1.600,1.13601,1.534,1.15093,1.50333,1.29066,1.304
3,2018-01-22,1.207,1.3513,1.04387,1.23145,1.20631,1.356,1.53408,1.309,1.25412,...,1.197,1.22494,1.31,1.590,1.13267,1.517,1.14493,1.50356,1.30146,1.318
4,2018-01-29,1.205,1.3627,1.04356,1.23084,1.21437,1.371,1.54783,1.333,1.25487,...,1.197,1.22950,1.31,1.602,1.13311,1.517,1.15300,1.52101,1.30390,1.330


## df_PB95_pct

> Calculating the rate of return %

In [21]:
# Calculating the rate of return:
df_PB95_pct = df_PB95.copy()

for col in df_PB95.columns[1:]:
    df_PB95_pct[col] = df_PB95[col].pct_change()
df_PB95_pct

,Date,Austria,Belgium,Bulgaria,Cyprus,Czechia,Germany,Denmark,Estonia,Spain,...,Luxembourg,Latvia,Malta,Netherlands,Poland,Portugal,Romania,Sweden,Slovenia,Slovakia
0,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-08,0.009213,0.009115,0.005367,0.006833,-0.001375,-0.001451,-0.008918,0.006467,0.005698,...,0.000000,0.000000,0.0,0.003173,0.003141,0.008609,0.019557,0.003867,0.000000,0.001535
2,2018-01-15,0.000000,0.003452,-0.000393,0.000771,0.005021,-0.017442,-0.000391,0.054618,0.005207,...,0.014407,0.027530,0.0,0.012018,0.004901,0.007223,0.010829,0.001866,0.001373,-0.000766
3,2018-01-22,0.001660,-0.011051,0.000882,-0.001103,0.007761,0.002959,0.000724,-0.003046,0.004107,...,0.000000,0.004560,0.0,-0.006250,-0.002940,-0.011082,-0.005213,0.000153,0.008368,0.010736
4,2018-01-29,-0.001657,0.008436,-0.000297,-0.000495,0.006682,0.011062,0.008963,0.018335,0.000598,...,0.000000,0.003723,0.0,0.007547,0.000388,0.000000,0.007048,0.011606,0.001875,0.009105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,2023-11-27,-0.015813,-0.016260,-0.008105,-0.009440,-0.005986,0.000552,-0.006914,-0.000618,-0.012053,...,-0.010575,-0.051051,0.0,-0.009142,-0.003234,-0.013506,-0.006578,0.001963,-0.002211,-0.001890
297,2023-12-04,-0.007712,-0.008286,-0.007115,-0.006882,-0.017286,-0.006064,-0.000133,-0.017316,-0.008987,...,0.000000,0.014229,0.0,-0.008201,-0.000390,-0.007738,-0.000803,0.010710,-0.007892,-0.010101
298,2023-12-11,-0.024611,-0.005836,-0.012464,-0.009873,-0.007549,-0.012202,-0.021324,0.023285,-0.008545,...,-0.024048,-0.001056,0.0,-0.013437,-0.005323,-0.014997,-0.008085,-0.027246,-0.010501,-0.001913
299,2023-12-18,-0.015272,-0.011888,-0.011347,-0.022900,-0.023752,-0.011791,0.000190,-0.004305,-0.009039,...,-0.017112,-0.007335,0.0,-0.011524,-0.011614,-0.008526,-0.001282,0.019226,-0.028848,-0.019808


In [ ]:
## Macierz korelacji:

# df_ON

> The same operations like in PB95

In [23]:
df_ON = df_PL[['Date']].copy()
df_ON = df_ON.reset_index(drop=True)  # Resetujemy indeksy również dla df_PB95

for key, dataframe in dfs.items():
    country_code = key.split("_")[1]  # Wyciąganie dwuliterowego kodu kraju z klucza (nazwy df, np. "df_AT")
    df_ON[country_code] = dataframe['Gas oil automobile'].reset_index(drop=True)  # Wyciągamu PB95 Resetujemy indeksy 

## Modyfikacje ON

In [24]:
#Zostawiamy tylko datetime wiersze w kolumnie date. Wyrzucamy 3 pierwsze wiersze
df_ON['Date'] = pd.to_datetime(df_ON['Date'], errors='coerce')
df_ON = df_ON.dropna(subset=['Date'])


In [25]:
# Removing commas from each value:
for col in df_ON.columns[1:]:
    df_ON[col] = df_ON[col].str.replace(',', '')

# Converting each column to float type:
for col in df_ON.columns[1:]:
    df_ON[col] = df_ON[col].astype(float, errors='ignore')

# Dividing everything by 1000 - to get the price per liter:
df_ON.iloc[:, 1:] = df_ON.iloc[:, 1:] / 1000


In [28]:
# Date range
df_ON = df_ON[(df_ON['Date'] >= Date_start) & (df_ON['Date'] <= Date_end)]

# Modifications
df_ON = df_ON.reset_index(drop=True)  # Resetting index
df_ON['Date'] = df_ON['Date'].dt.date  # Converting the 'Date' column to date format
df_ON = df_ON.iloc[::-1].reset_index(drop=True)

In [29]:
df_ON.head()

,Date,AT,BE,BG,CY,CZ,DE,DK,EE,ES,...,LU,LV,MT,NL,PL,PT,RO,SE,SI,SK
0,2018-01-01,1.146,1.3017,1.02781,1.23797,1.16209,1.207,1.32037,1.237,1.13764,...,1.058,1.09335,1.18,1.283,1.09939,1.321,1.13810,1.49302,1.23631,1.160
1,2018-01-08,1.157,1.3200,1.03431,1.24141,1.16859,1.214,1.30662,1.265,1.14615,...,1.058,1.09335,1.18,1.294,1.10499,1.335,1.16144,1.49992,1.23631,1.177
2,2018-01-15,1.156,1.3007,1.04505,1.24580,1.17242,1.190,1.30611,1.303,1.15564,...,1.058,1.12700,1.18,1.301,1.11188,1.342,1.17227,1.49794,1.23758,1.190
3,2018-01-22,1.159,1.2975,1.04106,1.25230,1.18119,1.190,1.29362,1.290,1.16362,...,1.058,1.13350,1.18,1.290,1.10808,1.322,1.16790,1.48780,1.24248,1.202
4,2018-01-29,1.155,1.3037,1.04704,1.25325,1.18722,1.197,1.29389,1.305,1.16575,...,1.058,1.13613,1.18,1.295,1.10837,1.320,1.17590,1.49719,1.24691,1.195


## df_ON_pct

In [30]:
# Liczenie stopy zwrotu:
df_ON_pct = df_ON.copy()

for col in df_ON.columns[1:]:
    df_ON_pct[col] = df_ON[col].pct_change()
df_ON_pct.head()  
    

,Date,AT,BE,BG,CY,CZ,DE,DK,EE,ES,...,LU,LV,MT,NL,PL,PT,RO,SE,SI,SK
0,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-08,0.009599,0.014059,0.006324,0.002779,0.005593,0.005800,-0.010414,0.022635,0.007480,...,0.0,0.000000,0.0,0.008574,0.005094,0.010598,0.020508,0.004622,0.000000,0.014655
2,2018-01-15,-0.000864,-0.014621,0.010384,0.003536,0.003277,-0.019769,-0.000390,0.030040,0.008280,...,0.0,0.030777,0.0,0.005410,0.006235,0.005243,0.009325,-0.001320,0.001027,0.011045
3,2018-01-22,0.002595,-0.002460,-0.003818,0.005218,0.007480,0.000000,-0.009563,-0.009977,0.006905,...,0.0,0.005768,0.0,-0.008455,-0.003418,-0.014903,-0.003728,-0.006769,0.003959,0.010084
4,2018-01-29,-0.003451,0.004778,0.005744,0.000759,0.005105,0.005882,0.000209,0.011628,0.001830,...,0.0,0.002320,0.0,0.003876,0.000262,-0.001513,0.006850,0.006311,0.003565,-0.005824


# df_LPG

In [33]:
# LPG
df_LPG = df_PL[['Date']].copy()
df_LPG = df_LPG.reset_index(drop=True)  

for key, dataframe in dfs.items():
    country_code = key.split("_")[1]  
    df_LPG[country_code] = dataframe['LPG'].reset_index(drop=True)  

## Modyfikacje LPG

In [36]:
# Keeping only datetime rows in the 'Date' column. Dropping the first 3 rows.
df_LPG['Date'] = pd.to_datetime(df_LPG['Date'], errors='coerce') 
df_LPG = df_LPG.dropna(subset=['Date'])  

In [37]:
# Removing commas from each value:
for col in df_LPG.columns[1:]:
    df_LPG[col] = df_LPG[col].str.replace(',', '')

# Converting each column to float type:
for col in df_LPG.columns[1:]:
    df_LPG[col] = df_LPG[col].astype(float, errors='ignore')

# Dividing everything by 1000 - to get the price per liter:
df_LPG.iloc[:, 1:] = df_LPG.iloc[:, 1:] / 1000
   

In [38]:
# Date range
df_LPG = df_LPG[(df_LPG['Date'] >= Date_start) & (df_LPG['Date'] <= Date_end)]

# Modifications
df_LPG = df_LPG.reset_index(drop=True)  # Resetting index
df_LPG['Date'] = df_LPG['Date'].dt.date  # Converting the 'Date' column to date format

# Reversing the order of rows
df_LPG = df_LPG.iloc[::-1].reset_index(drop=True)


In [40]:
# Dropping columns with all NaN values
df_LPG.dropna(axis=1, how='all', inplace=True)

In [41]:
df_LPG.head()

,Date,CZ,FR,LT,PL,SK
0,2018-01-01,0.55316,0.77727,0.54117,0.51454,0.586
1,2018-01-08,0.55426,0.81612,0.54094,0.51073,0.587
2,2018-01-15,0.55356,0.80757,0.54070,0.50768,0.586
3,2018-01-22,0.55593,0.80958,0.54082,0.50584,0.585
4,2018-01-29,0.55829,0.80887,0.53921,0.50673,0.584


## df_LPG_pct

In [ ]:
# Liczenie stopy zwrotu:
df_LPG_pct = df_LPG.copy()

for col in df_LPG.columns[1:]:
    df_LPG_pct[col] = df_LPG[col].pct_change()

# Output - excel 

In [42]:
# Save to excel
excel_path = os.path.join(output_folder, "Results" + Date_end + ".xlsx")

with pd.ExcelWriter(excel_path) as writer:
    # Najpierw zapisujemy df_PB95 jako pierwszy arkusz
    df_PB95.to_excel(writer, sheet_name='df_PB95', index=False)
    df_PB95_pct.to_excel(writer, sheet_name='df_PB95_pct', index=False)
    df_ON.to_excel(writer, sheet_name='df_ON', index=False)
    df_ON_pct.to_excel(writer, sheet_name='df_ON_pct', index=False)
    df_LPG.to_excel(writer, sheet_name='df_LPG', index=False)
    df_LPG_pct.to_excel(writer, sheet_name='df_LPG_pct', index=False)
    
    # Remaining DataFrames from the dictionary
    for key, dataframe in dfs.items():
        dataframe.to_excel(writer, sheet_name=key, index=False)

NameError: name 'output_folder' is not defined